# LangChain의 구성요소

1. `Model` IO은 input과 output으로 나누어져 있습니다.  입력(input)은 prompt(명령을 내리는 곳)입니다. 
    * prompts
    * Language Models
    * Output parser 

2. `Retrieval`은 외부 데이터로 접근하여 이를 모델에 어떻게 제공하는지에 관련된 것입니다.
    * document loaders
    * Transformers
    * Text Embedding 
    * Vector Stores
    * Retrievers 

3. `Chains` - 이전에 조금 해봤던 부분

4. `Agents`는 독립적으로 AI가 작동하도록 만들 수 있게 해주는 기능 (Chains이 필요한 도구들을 선택하여 사용할 수 있도록). 커스텀 도구를 만들어 준다면 chains 스스로 사용할 tool들을 선택할 수 있습니다. 

5. `Memory`는 그냥 챗봇이 기억할 수 있도록 해주는 기능 

6. `callbacks`은 기본적으로 model이 무엇을 하고있는지 중간에 알 수 있도록. 모델이 답변을 제공하기 전에 실제로 모델이 어떤 일을 하고 있는 지 확인할 수 있습니다. 

## FewShotPromptTemplate 

- Fewshot은 모델에게 예제들을 준다는 뜻과 같습니다. 더 나은 대답을 할 수 있도록 하는 예제들을 더 줄 수도 있죠. 
- 구체적으로 대답하는 AI 모델이 필요하다고 가정해봅시다. 모델에게 어떻게 대답해야 하는지에 대한 예제를 AI 모델에게 주는 것이 prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 훨씬 좋습니다. 
    예를 들어 모델에게 콤마를 써서 구분해달라고 하거나, 소문자만 써야한다거나 등등 처럼 이렇게 prompt로 주는 것보다 내가 원하는 것을 예제로 보여주는 것이 더 성공적입니다. 왜냐하면 모델이 텍스트를 만들기 때문이죠. 따라서 prompt로 명령하는 것보다 어떻게 대답해야하는지에 대한 예시를 보여주는 것이 더 나은 방법입니다. 이게 fewShotPromptTemplate가 하는 일입니다. 


이를 통해 예제를 형식화 할 수 있습니다. 예를 들어 고객지원 봇을 만든다고 해볼게요. 회사가 고객 게시판 같이 이미 고객 지원을 위한 시스템이 있다고 했을 때, 그곳에는 많은 고객들과의 대화 기록이나 다른 많은 기록들이 있을 것입니다. 그럼 language model에게 어떻게 고객들에게 대응하는지 알려주고 싶을 것입니다. 그럼 그냥 대화 기록 같은 것들을 데이터베이스에서 가져와서 FewShotPromptTemplate를 사용하여 형식화 시켜주면 더 빠르게 잘 만들 수 있을 것입니다. 이 방식이 기본적으로 언어 모델에게 예제를 주는 방식입니다.  

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n") # examples와 동일한 변수로 입력해야함

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix= "Human: What do you know about {country}?", # 위의 형식화가 끝나고 마지막에 실행 사용자의 질문이 어떻게 보여질지
    input_variables=["country"]    
)

chain = prompt | chat 

chain.invoke({
    'country': 'korea'
})                           # 결과가 예시와 같게 출력됨.

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

### FewShotChatMessagePromptTemplate

채팅 메세지를 위한 FewShotChatMessagePromptTemplate 


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
{
"country": "France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai","{answer}")
]) 

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert, you give short answers"), # 답변의 길이를 짧게 유지하기 위해 추가
    example_prompt,
    ("human",  "What do you know about {country}?"),
])

chain = final_prompt | chat 

chain.invoke({
    'country': 'Korea'
})                           # 결과가 예시와 같게 출력됨.

I know this:
There are two Koreas - North Korea and South Korea.
Capital of North Korea: Pyongyang
Capital of South Korea: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: North Korean Won (North Korea) and South Korean Won (South Korea)

AIMessageChunk(content='I know this:\nThere are two Koreas - North Korea and South Korea.\nCapital of North Korea: Pyongyang\nCapital of South Korea: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: North Korean Won (North Korea) and South Korean Won (South Korea)')

### LengthBasedExampleSelector 

예제들을 형식화할 수 있고 사용자가 설정한 세팅값에 따라 prompt에 알맞은 예제를 골라줄 것입니다.

In [8]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        return self.examples.append(example)     
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n") # examples와 동일한 변수로 입력해야함

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix= "Human: What do you know about {country}?", # 위의 형식화가 끝나고 마지막에 실행 사용자의 질문이 어떻게 보여질지
    input_variables=["country"]    
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\nHuman: What do you know about Brazil?'

### Serialization and Composition

json , yaml 파일 만들기

json 파일로 만든 프롬프트를 가져와서 사용할 수 있음. 
프롬프트 저장용.
prompt를 별도의 저장용으로 만들 수 있다.



In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate # PipelinePromptTemplate는 많은 prompt를 하나의 prompt로 합치는 것을 도와줌

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts,
                                     )

full_prompt.format(
    character = "Pirate",
    example_question = "What is your location?",
    example_answer = "Arrrg! That is a secret!",
    question = "What is your favorite food?",
)

chain = full_prompt | chat

chain.invoke({
    "character" : "Pirate",
    "example_question" : "What is your location?",
    "example_answer" : "Arrrg! That is a secret!",
    "question" : "What is your favorite food?",
})

Arrrg! Me favorite food be a good ol' plate o' fish and chips! The taste o' the salty sea in me mouth be a delight, matey!

AIMessageChunk(content="Arrrg! Me favorite food be a good ol' plate o' fish and chips! The taste o' the salty sea in me mouth be a delight, matey!")

### Caching 

캐싱을 사용하면 LM(언어 모델)의 응답을 저장할 수 있습니다. 

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(False)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller por

In [15]:
chat.predict("How do you make italian pasta") # 재실행하면 똑같은 프롬프트의 내용을 저장했다가 다시 알려줌. 비용 절약

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into smal

### Serialization

비용을 알아보자 

usage를 사용. 

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju?")
    b = chat.predict("What is the recipe for bread?")
    
    print(a,b,'\n')
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Soju is a traditional Korean distilled alcoholic beverage. Here is a simple recipe to make soju at home:\n\nIngredients:\n- 1 cup of rice\n- 1 cup of nuruk (Korean fermentation starter)\n- 10 cups of water\n- 1 cup of sugar\n\nInstructions:\n1. Rinse the rice thoroughly until the water runs clear.\n2. Soak the rice in water for about 1 hour.\n3. Drain the rice and transfer it to a large pot.\n4. Add 10 cups of water to the pot and bring it to a boil.\n5. Reduce the heat to low and simmer the rice for about 1 hour until it becomes soft and mushy.\n6. Remove the pot from heat and let it cool down to room temperature.\n7. Once the rice has cooled, add the nuruk to the pot and mix well.\n8. Cover the pot with a clean cloth and let it ferment for about 3-4 